In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import tensorflow as tf
import keras

device = "/GPU:0" if tf.config.list_physical_devices("GPU") else "/CPU:0"
print(device)

/CPU:0


In [2]:
# Load the dataset
df = pd.read_csv("../../../data/UNSW_NB15/UNSW_NB15.csv")
df = df.drop(labels=["id", "attack_cat"], axis=1)
df = df.dropna()

In [3]:
# Extract categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = numeric_cols.drop(['label'])

In [4]:
# Encode categorical variables
for col in categorical_cols:
    if col in df.columns:
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col])

In [5]:
# Drop label column
x = df.drop(['label'], axis=1)
y = df['label']

# Split the dataset
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y
)

In [6]:
# Apply standard scaling to numeric columns
scaler = StandardScaler()
x_train[numeric_cols] = scaler.fit_transform(x_train[numeric_cols])
x_test[numeric_cols] = scaler.transform(x_test[numeric_cols])

In [11]:
# Model definition
input_shape = (x_train.shape[1],)
n_classes = 2

model = keras.Sequential([
    # input layer
    keras.layers.Input(shape=input_shape),
    # hidden layers
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(96, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.25),
    # output layer
    keras.layers.Dense(n_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# Model training
with tf.device(device):
    model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 570us/step - accuracy: 0.8635 - loss: 0.3469
Epoch 2/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 557us/step - accuracy: 0.9051 - loss: 0.1866
Epoch 3/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 572us/step - accuracy: 0.9106 - loss: 0.1693
Epoch 4/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 565us/step - accuracy: 0.9150 - loss: 0.1623
Epoch 5/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 568us/step - accuracy: 0.9180 - loss: 0.1605
Epoch 6/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 575us/step - accuracy: 0.9191 - loss: 0.1574
Epoch 7/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 565us/step - accuracy: 0.9218 - loss: 0.1528
Epoch 8/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 558us/step - accuracy: 0.9241 - loss: 0.1513
Epoch 9/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 556us/step - accuracy: 0.9236 - loss: 0.1500
Epoch 10/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 4s 555us/step - accuracy: 0.9249 - loss: 0.1491


In [ ]:
# Make predictions
with tf.device(device):
    y_pred = model.predict(x_test)
    y_pred = np.argmax(y_pred, axis=1)

# Evaluate the model
metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "f1_score": f1_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred),
    "confusion_matrix": confusion_matrix(y_test, y_pred)
}
print(metrics)